[![GitHub stars](https://img.shields.io/github/stars/imcaspar/gpt2-ml?style=social)](https://github.com/imcaspar/gpt2-ml)
[![GitHub](https://img.shields.io/github/license/imcaspar/gpt2-ml)](https://github.com/imcaspar/gpt2-ml)
[![Twitter URL](https://img.shields.io/twitter/url?style=social&url=https%3A%2F%2Fgithub.com%2Fimcaspar%2Fgpt2-ml)](https://twitter.com/intent/tweet?text=Wow:&url=https://github.com/imcaspar/gpt2-ml)
### Instructions for running:

* Press the ▶️button on the left of each of the cells
* View the code: Double click any of the cells
* Hide the code: Double click the right side of the cell

In [ ]:
#@title #Prerequisites
#%tensorflow_version 1.x
!pip install -I tensorflow==1.15.2 &> tmp.log
!git clone -q https://github.com/imcaspar/gpt2-ml
%cd /content/gpt2-ml
!mkdir -p /content/gpt2-ml/models/mega

!perl 3rd/gdown.pl/gdown.pl https://drive.google.com/open?id=1bhZhjNSgV1CGI-PR2NDzNtda5FAwN29B models/mega/model.ckpt-220000.data-00000-of-00001
!wget -q --show-progress https://github.com/imcaspar/gpt2-ml/releases/download/v1.0/model.ckpt-220000.index -P models/mega
!wget -q --show-progress https://github.com/imcaspar/gpt2-ml/releases/download/v1.0/model.ckpt-220000.meta -P models/mega
!echo 'Download finished.'
# If gdown.pl failed, please uncomment following code & exec
# !python3 scripts/down_gdrive_file.py -file_id='1n_5-tgPpQ1gqbyLPbP1PwiFi2eo7SWw_' -file_path='models/mega/model.ckpt-100000.data-00000-of-00001'

In [ ]:
#@title #Inference
min_length = 150#@param {type:"number", min:5, max:1024, step:1}
sample_num =   5#@param {type:"number", min:1, max:50, step:1}
!PYTHONPATH=$(pwd) python3 scripts/interactive_conditional_samples.py -model_config_fn configs/mega.json -model_ckpt models/mega/model.ckpt-220000 -eos_token 102 -min_len $min_length -samples $sample_num